# Generic Rule Induction Notebook

Continuoulsy refined.

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment

### Configuration


In [154]:
CONFIG1 = { 
    'NAME' : 'binary-churn-quantile-brcg', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'BRCG',
    'TARGET_LABEL': 'CHURN',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': 'T'
     }
CONFIG2 = { 
    'NAME' : 'continuous-churn', 
    'DATA_SET': 'data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'pChurn',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': None
     }
CONFIG3 = { 
    'NAME' : 'bike-demand', 
    'DATA_SET': '../data/SeoulBikeData.csv',
    'DATA_TYPES': {'Rented Bike Count': float, 'Hour': float, 'Humidity': float, 'Visibility (10m)': float, 'RatePlan': str},
    'DROP': ['Date'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Rented Bike Count',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Dew point temperature(C)',
    'POS_CLASS': None
     }
CONFIG4 = { 
    'NAME' : 'heloc', 
    'DATA_SET': '../data/heloc.csv',
    'DATA_TYPES': {},
    'DROP': ['RiskPerformance'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Probabilities',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'ExternalRiskEstimate',
    'POS_CLASS': None
     }

CONFIG = CONFIG4
print('Proceed with configuration:', CONFIG['NAME'])

Proceed with configuration: heloc


In [155]:
import pandas as pd
import numpy as np
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
# from ripper.Ripper import Ripper
import time
import warnings

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
from aix360.algorithms.rbm import FeatureBinarizer, FeatureBinarizerFromTrees

# from explainer import Explainer

# TODO make imports dependent on CONFIG
# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [156]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG':
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert) 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10459 entries, 0 to 10458
Data columns (total 24 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ExternalRiskEstimate                10459 non-null  int64  
 1   MSinceOldestTradeOpen               10459 non-null  int64  
 2   MSinceMostRecentTradeOpen           10459 non-null  int64  
 3   AverageMInFile                      10459 non-null  int64  
 4   NumSatisfactoryTrades               10459 non-null  int64  
 5   NumTrades60Ever2DerogPubRec         10459 non-null  int64  
 6   NumTrades90Ever2DerogPubRec         10459 non-null  int64  
 7   PercentTradesNeverDelq              10459 non-null  int64  
 8   MSinceMostRecentDelq                10459 non-null  int64  
 9   MaxDelq2PublicRecLast12M            10459 non-null  int64  
 10  MaxDelqEver                         10459 non-null  int64  
 11  NumTotalTrades                      10459

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,Probabilities
0,55,144,4,84,20,3,0,83,2,3,...,0,0,0,33,-8,8,1,1,69,0.225887
1,61,58,15,41,2,4,4,100,-7,0,...,0,0,0,0,-8,0,-8,-8,0,0.041999
2,67,66,5,24,9,0,0,100,-7,7,...,0,4,4,53,66,4,2,1,86,0.055575
3,66,169,1,73,28,1,1,93,76,6,...,0,5,4,72,83,6,4,3,91,0.094626
4,81,333,27,132,12,0,0,100,-7,7,...,0,1,1,51,89,3,1,0,80,0.637285


In [157]:
df[CONFIG['TARGET_LABEL']].describe()

count    10459.000000
mean         0.478062
std          0.300649
min          0.000553
25%          0.194124
50%          0.449646
75%          0.765729
max          0.997082
Name: Probabilities, dtype: float64

### Train, Test Split

In [158]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (7321, 23) (7321,)
Test: (3138, 23) (3138,)


### Reference Performance

In [159]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

needs prior encoding of categoricals


### Binarization

In [160]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7321 entries, 528 to 7270
Columns: 310 entries, ('ExternalRiskEstimate', '<=', 56.0) to ('PercentTradesWBalance', '>', 100.0)
dtypes: int64(310)
memory usage: 17.4 MB


operation   <=                                            >                 \
value     56.0 61.0 65.0 68.0 71.0 74.0 78.0 81.0 86.0 56.0 61.0 65.0 68.0   
528          0    1    1    1    1    1    1    1    1    1    0    0    0   
1233         1    1    1    1    1    1    1    1    1    0    0    0    0   
779          0    0    0    0    0    0    0    0    1    1    1    1    1   
722          0    0    1    1    1    1    1    1    1    1    1    0    0   
5190         0    0    0    0    0    0    0    0    1    1    1    1    1   
6974         0    0    0    0    0    0    1    1    1    1    1    1    1   
1860         0    0    0    0    1    1    1    1    1    1    1    1    1   
9189         0    0    1    1    1    1    1    1    1    1    1    0    0   
8137         0    0    1    1    1    1    1    1    1    1    1    0    0   
3780         0    0    0    0    0    0    0    1    1    1    1    1    1   

operation                           
value     71.0 74.0 78.0 81.0 86.0  
528          0    0    0    0    0  
1233         0    0    0    0    0  
779          1    1    1    1    0  
722          0    0    0    0    0  
5190         1    1    1    1    0  
6974         1    1    0    0    0  
1860         0    0    0    0    0  
9189         0    0    0    0    0  
8137         0    0    0    0    0  
3780         1    1    1    0    0

### Rule Induction

In [161]:
start_time = time.time()

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    print('TODO plugin RIPPER here')
    # Ripper(d=2, k=2, pruning_threshold=50, mode='python')
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('training time: ' + str(end_time - start_time))

training time: 12.506050109863281


### Evaluation

In [162]:
if CONFIG['TYPE'] == 'BINARY':
    # compute predictive accuracy
    # print(estimator.dnf_rules.items())
    y_pred = estimator.predict(x_test_bin)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)
    # print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # print('Accuracy 2:', accuracy_score(y_test, estimator.predict(x_test)))
    # print('Accuracy 3', estimator.score(x_test, y_test))
    # print('Accuracy 4', estimator.score(x_test, y_test))

    # res = estimator.score(x_test, y_test) # this score function is defect
    # print('acc: ' + str(res))
    # print('rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
    # estimator.rule_map
    # print(estimator.rule_list_to_pretty_string())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')


R2 Score = 0.723625335279336
Explained Variance = 0.7236253352856212
Mean abs. error = 0.11259565331018143
Max error = 0.7583258001131231


In [163]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)

# uncomment the following line for a full optimized view of the model as data frame
# explanation.style

                                                 rule coefficient
0                                         (intercept)    0.593558
1   MSinceMostRecentInqexcl7days <= 0.00 AND MSinc...   -0.134943
2   ExternalRiskEstimate > 61.00 AND AverageMInFil...    0.131398
3   PercentTradesNeverDelq > 83.00 AND PercentInst...    0.104763
4   PercentTradesNeverDelq <= 96.00 AND MSinceMost...  -0.0923575
5                       ExternalRiskEstimate <= 74.00  -0.0882428
6   ExternalRiskEstimate <= 81.00 AND MSinceMostRe...   -0.085697
7   MSinceOldestTradeOpen > 98.00 AND NumSatisfact...   0.0732408
8   ExternalRiskEstimate <= 78.00 AND MSinceMostRe...   -0.071617
9   ExternalRiskEstimate <= 86.00 AND MSinceMostRe...  -0.0633187
10  AverageMInFile <= 81.00 AND MSinceMostRecentIn...  -0.0584207
11  AverageMInFile > 27.00 AND NumSatisfactoryTrad...   0.0559875
12                NetFractionRevolvingBurden <= 36.00   0.0468792
13                      ExternalRiskEstimate <= 71.00  -0.0441072
14        